# 5주차과제

***

## 1. Employee 데이터

- employee_train, employee_test
- employee_submission

### 1-1. Education 형태별로 Age의 중위값, 평균, 표준편차를 출력하시오.

In [2]:
import pandas as pd
import numpy as np

In [57]:
train = pd.read_csv('employee_train.csv')
test = pd.read_csv('employee_test.csv')
submission = pd.read_csv('employee_submission.csv')

In [4]:
train.head()

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot,employee_id
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0,1
1,Bachelors,2013,Pune,1,28,Female,No,3,1,2
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0,3
3,Masters,2016,Bangalore,3,27,Male,No,5,1,4
4,Masters,2017,Pune,3,24,Male,Yes,2,1,5


In [5]:
test.head()

,Unnamed: 0,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,employee_id
0,0,Bachelors,2015,Bangalore,3,28,Male,No,3,4001
1,1,Bachelors,2015,Pune,3,40,Female,Yes,2,4002
2,2,Masters,2018,New Delhi,1,24,Male,Yes,2,4003
3,3,Bachelors,2013,Pune,3,30,Male,No,4,4004
4,4,Masters,2017,New Delhi,3,29,Male,No,1,4005


In [6]:
train.groupby('Education').Age.agg([('MEDIAN', 'median'), ('MEAN', 'mean'), ('STD', 'std')])

,MEDIAN,MEAN,STD
Education,,,
Bachelors,28,29.379188,4.835172
Masters,28,29.259358,4.776628
PHD,28,29.121622,4.760817


### 1-2. Age에 대해 각각 표준화, 정규화를 수행한 후 각 결과에 대해 0.3 미만의 데이터 수를 출력하시오.

In [7]:
### 표준화
from sklearn.preprocessing import MinMaxScaler

In [8]:
scaler = MinMaxScaler()

In [10]:
train['scaled_age'] = scaler.fit_transform(train[['Age']])

In [13]:
train[train.scaled_age < 0.3].shape[0]

1885

In [14]:
### 정규화
from sklearn.preprocessing import StandardScaler

In [15]:
scaler = StandardScaler()

In [16]:
train['scaled_age'] = scaler.fit_transform(train[['Age']])

In [18]:
train[train.scaled_age < 0.3].shape[0]

2811

### 1-3. 여러분은 OO기업의 인사 담당자로 머신러닝을 활용해 직원들의 이직 여부(LeaveOrNot) 예측 모델링을 개발 중 입니다. 해당 문제에서 중요하게 사용할 수 있는 평가지표를 활용해 학습 및 검증을 수행하고 submission을 제출하시오.

- 하이퍼파라미터 튜닝 수행
- 3개 이상의 모델 사용

In [20]:
train.drop('scaled_age', axis = 1, inplace = True)

In [22]:
train.Education.unique()

array(['Bachelors', 'Masters', 'PHD'], dtype=object)

In [24]:
train.City.unique()

array(['Bangalore', 'Pune', 'New Delhi'], dtype=object)

In [25]:
train.Gender.unique()

array(['Male', 'Female'], dtype=object)

In [26]:
train.EverBenched.unique()

array(['No', 'Yes'], dtype=object)

In [29]:
train['Gender'] = [1 if g == 'Male' else 0 for g in train['Gender']]
test['Gender'] = [1 if g == 'Male' else 0 for g in test['Gender']]

In [30]:
train['EverBenched'] = [1 if e == 'Yes' else 0 for e in train['Gender']]
test['EverBenched'] = [1 if e == 'Yes' else 0 for e in test['Gender']]

In [32]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [34]:
train.drop('employee_id', axis = 1, inplace = True)
test.drop('employee_id', axis = 1, inplace = True)

In [37]:
test = test.iloc[:, 1:]

In [39]:
X = train.drop('LeaveOrNot', axis = 1)
y = train.LeaveOrNot

target = test[X.columns]

랜덤포레스트, lightgbm, xgboost

In [40]:
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [41]:
rf = RandomForestClassifier(random_state = 42)
lgbm = LGBMClassifier(random_state = 42)
xgb = XGBClassifier(random_state = 42)

In [42]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import precision_score

In [43]:
skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)

In [44]:
rf_params = {'max_depth' : [3, 5], 'n_estimators' : [100, 300]}
lgbm_params = {'max_depth' : [3, 5], 'n_estimators' : [100, 300], 'learning_rate' : [0.05, 0.01]}
xgb_params = {'max_depth' : [3, 5], 'n_estimators' : [100, 300], 'learning_rate' : [0.05, 0.01]}

In [47]:
rand_rf = RandomizedSearchCV(rf, rf_params, cv = skf, scoring = 'precision', n_jobs = -1)
rand_lgbm = RandomizedSearchCV(lgbm, lgbm_params, cv = skf, scoring = 'precision', n_jobs = -1)
rand_xgb = RandomizedSearchCV(xgb, xgb_params, cv = skf, scoring = 'precision', n_jobs = -1)

In [48]:
rand_rf.fit(X, y)
rand_lgbm.fit(X, y)
rand_xgb.fit(X, y)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifie

[22:23:01] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None,...ng=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_

In [49]:
rf = rand_rf.best_estimator_
lgbm = rand_lgbm.best_estimator_
xgb = rand_xgb.best_estimator_

In [62]:
rand_rf.best_params_

{'n_estimators': 300, 'max_depth': 5}

In [63]:
rand_lgbm.best_params_

{'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.01}

In [64]:
rand_xgb.best_params_

{'n_estimators': 300, 'max_depth': 3, 'learning_rate': 0.01}

In [67]:
final_pred = np.zeros((target.shape[0]))

for tr_idx, val_idx in skf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    rf = rand_rf.best_estimator_
    lgbm = rand_lgbm.best_estimator_
    xgb = rand_xgb.best_estimator_
    
    rf.fit(tr_x, tr_y)
    lgbm.fit(tr_x, tr_y)
    xgb.fit(tr_x, tr_y)
    
    val_rf = rf.predict_proba(val_x)[:, 1]
    val_lgbm = lgbm.predict_proba(val_x)[:, 1]
    val_xgb = xgb.predict_proba(val_x)[:, 1]
    
    val_rf = [1 if p >= 0.5 else 0 for p in val_rf]
    val_lgbm = [1 if p >= 0.5 else 0 for p in val_lgbm]
    val_xgb = [1 if p >= 0.5 else 0 for p in val_xgb]
    
    rf_score = precision_score(val_y, val_rf)
    lgbm_score = precision_score(val_y, val_lgbm)
    xgb_score = precision_score(val_y, val_xgb)
    print(f'RandomForest = {rf_score} / LightGBM = {lgbm_score} / XGBoost = {xgb_score}\n')
    
    ### PREDICTION
    
    fold_rf = rf.predict_proba(target)[:, 1] / 15
    fold_lgbm = lgbm.predict_proba(target)[:, 1] / 15
    fold_xgb = xgb.predict_proba(target)[:, 1] / 15
    
    final_pred += (fold_rf + fold_lgbm + fold_xgb)

/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:36:56] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
RandomForest = 0.8988095238095238 / LightGBM = 0.8978102189781022 / XGBoost = 0.8875



/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:36:57] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
RandomForest = 0.8681318681318682 / LightGBM = 0.8609271523178808 / XGBoost = 0.8367346938775511



/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:36:57] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
RandomForest = 0.9575757575757575 / LightGBM = 0.9290780141843972 / XGBoost = 0.9127906976744186



/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:36:58] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
RandomForest = 0.9337748344370861 / LightGBM = 0.8705035971223022 / XGBoost = 0.8705882352941177



/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:36:59] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
RandomForest = 0.9577464788732394 / LightGBM = 0.9206349206349206 / XGBoost = 0.9183673469387755



In [52]:
cls = [1 if p >= 0.5 else 0 for p in final_pred]

In [58]:
submission['LeaveOrNot'] = cls

In [61]:
submission.to_csv('result.csv', index = False)

***
## 2. Marketing Campaign 데이터
- marketing_train, markteing_test
- marketing_submission

### 2-1. 학습데이터의 MntFishProducts의 이상치의 개수를 구하시오.

In [147]:
train = pd.read_csv('marketing_train.csv')
test = pd.read_csv('marketing_test.csv')
submission = pd.read_csv('marketing_submission.csv')

In [70]:
train.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,172,88,88,3,8,10,4,7,0,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,2,1,6,2,1,1,2,5,0,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,111,21,42,1,8,2,10,4,0,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,10,3,5,2,2,0,4,6,0,0
4,7446,1967,Master,Together,62513.0,0,1,09-09-2013,16,520,...,0,42,14,2,6,4,10,6,0,0


In [71]:
test.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain
0,9239,1961,Master,Married,61923.0,0,2,26-07-2013,94,92,...,18,3,3,6,1,2,1,4,3,0
1,6431,1964,PhD,Married,45759.0,1,1,23-02-2013,13,42,...,18,3,0,4,2,2,0,3,7,0
2,5758,1982,Graduation,Married,65169.0,0,0,14-01-2014,23,1074,...,69,0,0,46,1,10,4,13,6,0
3,10858,1975,Graduation,Together,76532.0,1,1,09-09-2013,38,355,...,177,90,138,30,4,9,5,7,5,0
4,2569,1987,Graduation,Married,17323.0,0,0,10-10-2012,38,3,...,17,6,1,5,1,1,0,3,8,0


In [72]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   1800 non-null   int64  
 1   Year_Birth           1800 non-null   int64  
 2   Education            1800 non-null   object 
 3   Marital_Status       1800 non-null   object 
 4   Income               1783 non-null   float64
 5   Kidhome              1800 non-null   int64  
 6   Teenhome             1800 non-null   int64  
 7   Dt_Customer          1800 non-null   object 
 8   Recency              1800 non-null   int64  
 9   MntWines             1800 non-null   int64  
 10  MntFruits            1800 non-null   int64  
 11  MntMeatProducts      1800 non-null   int64  
 12  MntFishProducts      1800 non-null   int64  
 13  MntSweetProducts     1800 non-null   int64  
 14  MntGoldProds         1800 non-null   int64  
 15  NumDealsPurchases    1800 non-null   i

In [73]:
q1 = np.percentile(train['MntFishProducts'], 25)
q3 = np.percentile(train['MntFishProducts'], 75)
iqr = q3 - q1

In [74]:
q1, q3

(3.0, 50.0)

In [75]:
l_b = q1 - 1.5 * iqr
u_b = q3 + 1.5 * iqr

In [77]:
train[(train.MntFishProducts < l_b) | (train.MntFishProducts > u_b)].shape[0]

181

### 2-2. 주말에 마케팅 반응(Response)이 나타난 고객의 수를 구하시오.

In [89]:
train['Dt_Customer'] = train['Dt_Customer'].str[-4:] + '-' + train['Dt_Customer'].str[3:5] + '-' + train['Dt_Customer'].str[:2]

In [105]:
train['Dt_Customer'] = pd.to_datetime(train['Dt_Customer'])

In [106]:
train

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-04-09,58,635,...,172,88,88,3,8,10,4,7,0,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-08-03,38,11,...,2,1,6,2,1,1,2,5,0,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,111,21,42,1,8,2,10,4,0,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-10-02,26,11,...,10,3,5,2,2,0,4,6,0,0
4,7446,1967,Master,Together,62513.0,0,1,2013-09-09,16,520,...,0,42,14,2,6,4,10,6,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,10870,1967,Graduation,Married,61223.0,0,1,2013-06-13,46,709,...,42,118,247,2,9,3,4,5,0,0
1796,4001,1946,PhD,Together,64014.0,2,1,2014-10-06,56,406,...,0,0,8,7,8,2,5,7,0,0
1797,7270,1981,Graduation,Divorced,56981.0,0,0,2014-01-25,91,908,...,32,12,24,1,2,3,13,6,0,0
1798,8235,1956,Master,Together,69245.0,0,1,2014-01-24,8,428,...,80,30,61,2,6,5,10,3,0,0


In [95]:
train['Day'] = train.Dt_Customer.dt.day_name().str[:2]

In [97]:
train['weekend'] = [1 if day in ['Sa', 'Su'] else 0 for day in train['Day']]

In [102]:
train[train.weekend == 1].Response.value_counts()[1]

60

### 2-3. Income에 대해 표준화를 실시하고 0.2 이상 0.35 미만인 데이터에 속한 고객의 평균 나이를 구하시오.
- 날짜상 현재 년도은 2014년입니다. ex> Year_Birth가 1957이면 57세로 간주.

In [148]:
train['age'] = 2014 - train['Year_Birth']

In [149]:
test['age'] = 2014 - test['Year_Birth']

In [123]:
train.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response,scaled_age,scaled_income,age
0,5524,1957,Graduation,Single,58138.0,0,0,2012-04-09,58,635,...,3,8,10,4,7,0,1,0.087207,0.087207,57
1,2174,1954,Graduation,Single,46344.0,1,1,2014-08-03,38,11,...,2,1,1,2,5,0,0,0.069516,0.069516,60
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,1,8,2,10,4,0,0,0.107420,0.107420,49
3,6182,1984,Graduation,Together,26646.0,1,0,2014-10-02,26,11,...,2,2,0,4,6,0,0,0.039969,0.039969,30
4,7446,1967,Master,Together,62513.0,0,1,2013-09-09,16,520,...,2,6,4,10,6,0,0,0.093770,0.093770,47


In [108]:
from sklearn.preprocessing import MinMaxScaler

In [124]:
scaler = MinMaxScaler()

In [125]:
train.fillna(0, inplace = True)

In [126]:
train['scaled_income'] = scaler.fit_transform(train[['Income']])

In [127]:
train[(train.scaled_income >= 0.2) & (train.scaled_income < 0.35)].age.mean()

42.142857142857146

### 2-4. 여러분은 OO기업의 마케팅 담당자 입니다. 마케팅 효과 분석을 위해 고객 데이터를 분석 중으로 마케팅 반응 예측 모델링을 하고 있습니다. 마케팅 효과 분석에 적합한 평가지표를 설정하고 이를 바탕으로 모델을 학습 및 검증하여 최종 submission을 제출하시오.
- 0, 1의 이진 형태로 제출
- 3개 이상의 모델 사용
- 최소 1개 이상의 파생변수 생성
- 최소 1개 이상의 모델에 파라미터 튜닝 수행

In [129]:
train.drop(['scaled_age', 'scaled_income'], axis = 1, inplace = True)

In [150]:
train.drop(['ID', 'Year_Birth', 'Dt_Customer'], axis = 1, inplace = True)
test.drop(['ID', 'Year_Birth', 'Dt_Customer'], axis = 1, inplace = True)

In [151]:
train.Education.unique()

array(['Graduation', 'Master', 'PhD', 'Basic', '2n Cycle'], dtype=object)

In [152]:
test.Education.unique()

array(['Master', 'PhD', 'Graduation', '2n Cycle', 'Basic'], dtype=object)

In [153]:
train.Marital_Status.unique()

array(['Single', 'Together', 'Divorced', 'Married', 'Widow', 'Alone',
       'Absurd', 'YOLO'], dtype=object)

In [154]:
test.Marital_Status.unique()

array(['Married', 'Together', 'Single', 'Divorced', 'Widow', 'Absurd'],
      dtype=object)

In [155]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [156]:
train.shape, test.shape

((1800, 31), (440, 28))

In [158]:
test[['Marital_Status_Alone', 'Marital_Status_YOLO']] = 0

In [159]:
X = train.drop('Response', axis = 1)
y = train.Response

target = test[X.columns]

In [160]:
rf = RandomForestClassifier(random_state = 42)
lgbm = LGBMClassifier(random_state = 42)
xgb = XGBClassifier(random_state = 42)

In [161]:
from sklearn.metrics import recall_score

In [162]:
skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

In [168]:
lgbm_pred = np.zeros((target.shape[0]))
for tr_idx, val_idx in skf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    lgbm = LGBMClassifier(random_state = 42)
    lgbm.fit(tr_x, tr_y)
    
    val_pred = lgbm.predict_proba(val_x)[:, 1]
    val_pred = [1 if p >= 0.5 else 0 for p in val_pred]
    score = recall_score(val_y, val_pred)
    print(score)
    fold_pred = lgbm.predict_proba(target)[:, 1] / 10
    lgbm_pred += fold_pred

0.35714285714285715
0.32142857142857145
0.32142857142857145
0.35714285714285715
0.5
0.32142857142857145
0.4642857142857143
0.48148148148148145
0.3333333333333333
0.18518518518518517


In [169]:
submission['Response'] = [1 if p >= 0.5 else 0 for p in lgbm_pred]

In [171]:
submission.Response.value_counts()

0    404
1     36
Name: Response, dtype: int64